# Market Simulation

This notebook creates four files.
For a given number of market situations, each market situation is simulated with the same starting condition multiple times for a specified time horizon.
For every simulation, we note the arrival of customers in the `arrival_out` file, the purchases in the `buying_out` file and the current price for each company in the `traject_out` file. In the `demand_learning_data` file we save explanatory variables for market situations and if there was a purchase in the current situation.

In [1]:
import random
import numpy as np
import csv

In [2]:
def start():
    for i in range(iterateDifferentStart):
        createBasicInfo()

In [3]:
def createBasicInfo():
    pricesCompanies = [round(random.uniform(6,16),2) for _ in range(numberPlayers)]
    qualityCompanies = [round(random.uniform(1,5)) for _ in range(numberPlayers)]
    ratingCompanies = [round(random.uniform(90,100),1) for _ in range(numberPlayers)]
    
    priceRanks = [1 +  
              sum([1 for i in range(numberPlayers) if pricesCompanies[j]> pricesCompanies[i] and i!=j]) +
              sum([1 for i in range(numberPlayers) if pricesCompanies[j]== pricesCompanies[i] and i!=j])/2
              for j in range(numberPlayers)]

    qualityRanks = [1 +  
              sum([1 for i in range(numberPlayers) if qualityCompanies[j]> qualityCompanies[i] and i!=j]) +
              sum([1 for i in range(numberPlayers) if qualityCompanies[j]== qualityCompanies[i] and i!=j])/2
              for j in range(numberPlayers)]

    ratingRanks = [1 +  
              sum([1 for i in range(numberPlayers) if ratingCompanies[j]> ratingCompanies[i] and i!=j]) +
              sum([1 for i in range(numberPlayers) if ratingCompanies[j]== ratingCompanies[i] and i!=j])/2
              for j in range(numberPlayers)]

    accumulatedProfit = [0 for _ in range(numberPlayers)]
    createSimulationParameters(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit)

In [4]:
def calculateTimeOfAdjustmentsCompany(numberReactionEvent, company, timeBetweenMarketEvents):
    if numberReactionEvent==0:
        return(random.uniform(0,1)) 
    else: return(calculateTimeOfAdjustmentsCompany(numberReactionEvent-1, company, timeBetweenMarketEvents)+timeBetweenMarketEvents[company][numberReactionEvent])
    
def calculateTimeOfCustomerArrival(numberCustomerEvent, timeBetweenCustomerEvents):
    if numberCustomerEvent==0:
        return(0)
    else:
        return(calculateTimeOfCustomerArrival(numberCustomerEvent-1, timeBetweenCustomerEvents) + timeBetweenCustomerEvents[numberCustomerEvent])

In [5]:
def createSimulationParameters(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit):
    timeBetweenMarketEvents = [[random.uniform(0.8, 1.2) if numberReactionEvents != 1 else 1.1 for _ in range(numberReactionEvents)] for _ in range(numberPlayers)]


    timeOfAdjustmentsCompany = np.array([[calculateTimeOfAdjustmentsCompany(i, j, timeBetweenMarketEvents) for i in range(numberReactionEvents)] for j in range(numberPlayers)])

    priceAdjustmentsBeforeHorizon = set([timeOfAdjustmentsCompany.flatten()[i] for i in range(len(timeOfAdjustmentsCompany.flatten())) if timeOfAdjustmentsCompany.flatten()[i] < timeHorizon])
    priceAdjustmentsBeforeHorizon.add(timeHorizon)
    priceAdjustmentsBeforeHorizon = sorted(priceAdjustmentsBeforeHorizon)

    numberPriceAdjustmentsBeforeHorizon = len(priceAdjustmentsBeforeHorizon)

    timePriceAdjustments = [priceAdjustmentsBeforeHorizon[i] for i in range(numberPriceAdjustmentsBeforeHorizon)]

    # we have to be careful because of the first timeslot
    timeBetweenAdjustments = [timePriceAdjustments[i]-timePriceAdjustments[i-1] if i > 0 else timePriceAdjustments[i] for i in range(numberPriceAdjustmentsBeforeHorizon)]

    timeBetweenCustomerEvents = [meanDifferenceCustomerArrival*np.random.exponential() for _ in range(numberCustomerEvents)]

    timeOfCustomerArrival = [calculateTimeOfCustomerArrival(i, timeBetweenCustomerEvents) for i in range(numberCustomerEvents)]    
    
    companyAdjustingAtTime = [min(np.where(timeOfAdjustmentsCompany==timePriceAdjustments[i])[0])if timePriceAdjustments[i]!=timeHorizon else -1  for i in range(numberPriceAdjustmentsBeforeHorizon)]    
    
    randomCustomerScoringOfCompany = [[0 for _ in range(numberCustomerEvents)] for _ in range(numberPlayers)]

    randomCustomerScoringQuality = [random.uniform(0,1) for _ in range(numberCustomerEvents)]

    randomCustomerScoringRating = [random.uniform(0,0.5) for _ in range(numberCustomerEvents)]

    randomCustomerDecision = [None for _ in range(numberCustomerEvents)]

    
    for j in range(iterateSameStart): {
        createWatchParameters(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit,
                     timeBetweenMarketEvents, timeOfAdjustmentsCompany, priceAdjustmentsBeforeHorizon, numberPriceAdjustmentsBeforeHorizon,
                     timePriceAdjustments, timeBetweenAdjustments, timeBetweenCustomerEvents, timeOfCustomerArrival, companyAdjustingAtTime,
                     randomCustomerScoringOfCompany, randomCustomerScoringQuality, randomCustomerScoringRating, randomCustomerDecision)
    }

In [46]:
def createWatchParameters(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit,
                     timeBetweenMarketEvents, timeOfAdjustmentsCompany, priceAdjustmentsBeforeHorizon, numberPriceAdjustmentsBeforeHorizon,
                     timePriceAdjustments, timeBetweenAdjustments, timeBetweenCustomerEvents, timeOfCustomerArrival, companyAdjustingAtTime,
                     randomCustomerScoringOfCompany, randomCustomerScoringQuality, randomCustomerScoringRating, randomCustomerDecision):
    currentTimeProcess = 0
    countAdjustmentsCompany = 0
    watchedCompany = companyToWatch

    countAdjustmentsWatchedCompany = sum([1 for i in range(numberPriceAdjustmentsBeforeHorizon) if companyAdjustingAtTime[i] == watchedCompany])

    timeAdjustmentsWatchedCompany = [0 for _ in range(countAdjustmentsWatchedCompany+1)]
    timeBetweenAdjustmentsWatchedCompany = [timeAdjustmentsWatchedCompany[i] - timeAdjustmentsWatchedCompany[i-1] for i in range(1, countAdjustmentsWatchedCompany)]

    salesBetweenAdjustments = [0 for _ in range(countAdjustmentsWatchedCompany)]

    price = [pricesCompanies[watchedCompany] for _ in range(countAdjustmentsWatchedCompany+1)]
    quality = [qualityCompanies[watchedCompany] for _ in range(countAdjustmentsWatchedCompany+1)]
    rating = [ratingCompanies[watchedCompany] for _ in range(countAdjustmentsWatchedCompany+1)]

    pricesCompetitors = [[pricesCompanies[c] for _ in range(countAdjustmentsWatchedCompany+1)] for c in range(numberPlayers)]
    qualityCompetitors = [[qualityCompanies[c] for _ in range(countAdjustmentsWatchedCompany+1)] for c in range(numberPlayers)]
    ratingCompetitors = [[ratingCompanies[c] for _ in range(countAdjustmentsWatchedCompany+1)] for c in range(numberPlayers)]

    runSimulation(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit,
                     timeBetweenMarketEvents, timeOfAdjustmentsCompany, priceAdjustmentsBeforeHorizon, numberPriceAdjustmentsBeforeHorizon,
                     timePriceAdjustments, timeBetweenAdjustments, timeBetweenCustomerEvents, timeOfCustomerArrival, companyAdjustingAtTime,
                     randomCustomerScoringOfCompany, randomCustomerScoringQuality, randomCustomerScoringRating, randomCustomerDecision,
                 currentTimeProcess, countAdjustmentsCompany, watchedCompany, countAdjustmentsWatchedCompany, timeAdjustmentsWatchedCompany,
                 timeBetweenAdjustmentsWatchedCompany, salesBetweenAdjustments, price, quality, rating, pricesCompetitors, qualityCompetitors,
                 ratingCompetitors)

In [49]:
def runSimulation(pricesCompanies, qualityCompanies, ratingCompanies, priceRanks, qualityRanks, ratingRanks, accumulatedProfit,
                     timeBetweenMarketEvents, timeOfAdjustmentsCompany, priceAdjustmentsBeforeHorizon, numberPriceAdjustmentsBeforeHorizon,
                     timePriceAdjustments, timeBetweenAdjustments, timeBetweenCustomerEvents, timeOfCustomerArrival, companyAdjustingAtTime,
                     randomCustomerScoringOfCompany, randomCustomerScoringQuality, randomCustomerScoringRating, randomCustomerDecision,
                 currentTimeProcess, countAdjustmentsCompany, watchedCompany, countAdjustmentsWatchedCompany, timeAdjustmentsWatchedCompany,
                 timeBetweenAdjustmentsWatchedCompany, salesBetweenAdjustments, price, quality, rating, pricesCompetitors, qualityCompetitors,
                 ratingCompetitors):
    arrival_out = [[0 for _ in range(numberCustomerEvents)] for _ in range(numberPriceAdjustmentsBeforeHorizon-1)]
    buying_out = [[0 for _ in range(numberCustomerEvents)] for _ in range(numberPriceAdjustmentsBeforeHorizon-1)]
    trajekt1_out = [[0 for _ in range(numberPlayers)] for _ in range(numberPriceAdjustmentsBeforeHorizon-1)]

    for i in range(1,numberPriceAdjustmentsBeforeHorizon): 
        for customer in [customer for customer in range(numberCustomerEvents) if timePriceAdjustments[i-1]< timeOfCustomerArrival[customer] < timePriceAdjustments[i]]:
            arrival_out[i-1][customer]=[timeOfCustomerArrival[customer],0,customer]
            for company in range(numberPlayers):
                randomCustomerScoringOfCompany[company][customer]= (pricesCompanies[company]+ 
                randomCustomerScoringQuality[customer] * qualityCompanies[company] +
                randomCustomerScoringRating[customer] * (100-ratingCompanies[company]))
            if min([randomCustomerScoringOfCompany[company][customer] for company in range(numberPlayers)]) < random.uniform(5,15):
                randomCustomerDecision[customer] = np.argmin([randomCustomerScoringOfCompany[company][customer] for company in range(numberPlayers)])
                accumulatedProfit[randomCustomerDecision[customer]] = accumulatedProfit[randomCustomerDecision[customer]] + pricesCompanies[randomCustomerDecision[customer]]
                buying_out[i-1][customer]=[timeOfCustomerArrival[customer], pricesCompanies[randomCustomerDecision[customer]], customer, randomCustomerDecision[customer]]
        currentTimeProcess= priceAdjustmentsBeforeHorizon[i]
        for company in range(numberPlayers):
            trajekt1_out[i-1][company] = [pricesCompanies[company], currentTimeProcess, company]
        if currentTimeProcess < timeHorizon:
            pricesCompanies[companyAdjustingAtTime[i]] = priceAdjustmentRules(companyAdjustingAtTime[i], pricesCompanies)
            priceRanks = [1 +  
                  sum([1 for i in range(numberPlayers) if pricesCompanies[j]> pricesCompanies[i] and i!=j]) +
                  sum([1 for i in range(numberPlayers) if pricesCompanies[j]== pricesCompanies[i] and i!=j])/2
                  for j in range(numberPlayers)]
        '''if random.uniform(0,1)<0.2:
            qualityCompanies[companyAdjustingAtTime[i]] = round(random.uniform(0.5,5.5))
            ratingCompanies[companyAdjustingAtTime[i]] = round(random.uniform(90,100),1)
            qualityRanks = [1 +  
                  sum([1 for i in range(numberPlayers) if qualityCompanies[j]> qualityCompanies[i] and i!=j]) +
                  sum([1 for i in range(numberPlayers) if qualityCompanies[j]== qualityCompanies[i] and i!=j])/2
                  for j in range(numberPlayers)]

            ratingRanks = [1 +  
                  sum([1 for i in range(numberPlayers) if ratingCompanies[j]> ratingCompanies[i] and i!=j]) +
                  sum([1 for i in range(numberPlayers) if ratingCompanies[j]== ratingCompanies[i] and i!=j])/2
                  for j in range(numberPlayers)]'''
        if companyAdjustingAtTime[i]==watchedCompany:
            countAdjustmentsCompany = countAdjustmentsCompany+1
            timeAdjustmentsWatchedCompany[countAdjustmentsCompany]=currentTimeProcess
            price[countAdjustmentsCompany] = pricesCompanies[watchedCompany]
            quality[countAdjustmentsCompany] = qualityCompanies[watchedCompany]
            rating[countAdjustmentsCompany] = ratingCompanies[watchedCompany]
            salesBetweenAdjustments[countAdjustmentsCompany-1]= sum([1 for c in range(numberCustomerEvents) if timeAdjustmentsWatchedCompany[countAdjustmentsCompany-1]<timeOfCustomerArrival[c]<timeAdjustmentsWatchedCompany[countAdjustmentsCompany] and watchedCompany==randomCustomerDecision[c]])
            for c in range(numberPlayers):
                pricesCompetitors[c][countAdjustmentsCompany]= pricesCompanies[c]
                qualityCompetitors[c][countAdjustmentsCompany]= qualityCompanies[c]
                ratingCompetitors[c][countAdjustmentsCompany]= ratingCompanies[c]
    #print(salesBetweenAdjustments)

    with open('arrival_out_S3.csv', 'a') as file:
        writer = csv.writer(file, delimiter='\t')
        for w in range(numberPriceAdjustmentsBeforeHorizon-1):
            currentSim = list(filter(lambda x : x != 0, arrival_out[w]))
            for c in range(len(currentSim)):
                line = list(currentSim[c])
                writer.writerow(line)

    with open('buying_out_S3.csv', 'a') as file:
        writer = csv.writer(file, delimiter='\t')
        for w in range(numberPriceAdjustmentsBeforeHorizon-1):
            currentSim = list(filter(lambda x : x != 0, buying_out[w]))
            for c in range(len(currentSim)):
                line = list(currentSim[c])
                writer.writerow(line)

    with open('traject1_out_S3.csv', 'a') as file:
        writer = csv.writer(file, delimiter='\t')
        for w in range(numberPriceAdjustmentsBeforeHorizon-1):
            currentSim = list(filter(lambda x : x != 0, trajekt1_out[w]))
            for k in range(len(currentSim)):
                line= list(currentSim[k])
                writer.writerow(line)
                
    def explanatoryVariables(explanatoryVariable, countAdjustment):
        return {
            '0' : 1,
            '1' : priceRanksFirm[countAdjustment-1],
            '2' : 1 if priceRanksFirm[countAdjustment-1]==1 else 0,
            '3' : qualityRanksFirm[countAdjustment-1],
            '4' : ratingRanks[countAdjustment-1][watchedCompany],
            '5' : price[countAdjustment],
            '6' : price[countAdjustment] - min([pricesCompetitors[j][countAdjustment] for j in range(numberPlayers) if j!=watchedCompany]) if numberPlayers > 1 else 0,
            '7' : quality[countAdjustment],
            '8' : rating[countAdjustment],
            '9' : 15 - (price[countAdjustment]+0.5*quality[countAdjustment]+0.25*(100-rating[countAdjustment])) if(
            (price[countAdjustment]+0.5*quality[countAdjustment]+0.25*(100-rating[countAdjustment])) <
            min([pricesCompetitors[j][countAdjustment] +0.5*qualityCompetitors[j][countAdjustment]+0.25*(100-ratingCompetitors[j][countAdjustment]) for j in range(numberPlayers) if j!= watchedCompany]))
            else 0,
        }[str(explanatoryVariable)]

    priceRanksFirm = [1 +  
              sum([1 for i in range(numberPlayers) if price[j]> pricesCompetitors[i][j] and i!=watchedCompany]) +
              sum([1 for i in range(numberPlayers) if price[j]== pricesCompetitors[i][j] and i!=watchedCompany])/2
              for j in range(1,countAdjustmentsWatchedCompany+1)]

    qualityRanksFirm = [1 +  
              sum([1 for i in range(numberPlayers) if quality[j]> qualityCompetitors[i][j] and i!=watchedCompany]) +
              sum([1 for i in range(numberPlayers) if quality[j]== qualityCompetitors[i][j] and i!=watchedCompany])/2
              for j in range(1,countAdjustmentsWatchedCompany+1)]

    
    ratingRanks = [[1 +  
              sum([1 for i in range(numberPlayers) if ratingCompetitors[j][count]> ratingCompetitors[i][count] and i!=j]) +
              sum([1 for i in range(numberPlayers) if ratingCompetitors[j][count]== ratingCompetitors[i][count] and i!=j])/2
              for j in range(numberPlayers)]for count in range(1,countAdjustmentsWatchedCompany+1)]
    with open('demand_learning_data_S3.csv', 'a') as file:
        writer = csv.writer(file, delimiter='\t')
        for w in range(1, countAdjustmentsWatchedCompany+1):
            global currentLine
            currentLine = currentLine +1
            line = list([currentLine])
            for k in range(1,10):
                line.append(explanatoryVariables(k, w))
            line.append(salesBetweenAdjustments[w-1])
            writer.writerow(line)

In [50]:
def priceAdjustmentRules(c, pricesCompanies):
    return {
        '0' : 10 if pricesCompanies[c]<5 else round(min([max(4.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),2),
        '1' : 9 if pricesCompanies[c]<4 else round(min([max(3.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),2),
        '2' : 12 if pricesCompanies[c]<6 else round(min([max(5.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),2),
        '3' : round(random.uniform(11,11),2),
        '4' : round(random.uniform(13,13),2)
    }[str(c)]

numberPlayers = 5
timeHorizon = 100
numberReactionEvents = timeHorizon + 100
meanDifferenceCustomerArrival = 1
numberCustomerEvents = int(timeHorizon / meanDifferenceCustomerArrival + 100)
iterateDifferentStart = 100
iterateSameStart = 100
companyToWatch = 0
currentLine = 1
start()

200
201
200
201
200
201
200
201
200
201
200
201
200
201
200
201
200
201
200
201
200
201
200
201


KeyboardInterrupt: 